<h1><center>Hotel Booking Demand</center></h1>

## `Business Problem`

**Project Stakeholders**:
<br>
Stakeholders yang akan menggunakan model ini adalah:
1. Marketing Departement
2. Finance Departement

**Context**:
<br>
Sebuah perusahaan yang bergerak dibidang hospitality industry-khususnya perhotelan-dan berlokasi di Portugal. Hospitality industry lebih fokus dengan kepuasan pelanggan. Berdasarkan [The Big Four](https://www.hospitalitynet.org/opinion/4082318.html), ada bermacam-macam hospitality industry, seperti Food and Beverages, Travel and Tourism, Lodging, dan Recreation. Industri hotel termasuk ke dalam lodging, salah satu permasalahan yang sering terjadi pada industri hotel adalah pembatalan pesanan oleh pelanggan.

Target (is_canceled):

1 : pesanan yang dibatalkan (positif)

0 : pesanan yang **tidak** dibatalkan (negatif)

**Prolem Statement**:
<br>
Perusahaan ingin mengetahui atau memprediksi apakah suatu pelanggan akan membatalkan pesanan atau tidak. Hal ini dimaksudkan karena pembatalan pesanan dapat memberikan dampak negatif bagi revenue perusahaan dan reputasi perusahaan tersebut, yang akan berujung pada menurunnya performa bisnis.

Dengan mengidentifikasi pesanan mana yang kemungkinan dibatalkan, pihak hotel dapat menawarkan layanan lainnya seperti, diskon, tiket masuk ke pertunjukkan/taman hiburan, ataupun hal lainnya. Akan tetapi, penawaran-penawaran ini mungkin tidak dapat selalu diterapkan mengingatkan tingkat keekonomian/kesensitifan yang pelanggan berbeda-beda, contoh tamu perusahaan (coorporate guests).

**Goals**:
<br>
Sebisa mungkin perusahaan tidak kehilangan potential customer/pelanggan yang berujung pada meningkatnya revenue perusahaan.

**Analytic Approach**:
<br>
Jadi yang akan kita lakukan adalah menganalisis data untuk menemukan pola yang membedakan pelanggan yang akan membatalkan pesanan atau tidak.

Kemudian kita akan membangun model `klasifikasi` yang akan membantu perusahaan untuk dapat memprediksi pelanggan yang akan membatalkan pesanan atau tidak.

**Metric Evaluation**:
<br>
Type I Error: False Positive
<br>
Kondisi di mana pelanggan terprediksi **membatalkan** pesanan padahal kenyataannya tidak membatalkan pesanan

Type II Error: False Negative
<br>
Kondisi di mana pelangga terprediksi tidak membatalkan pesanan padahal kenyataannya membatalkan pesanan

Berdasarkan konsekuensinya, maka sebisa mungkin yang akan kita lakukan adalah membuat model yang dapat memprediksi sebanyak mungkin kelas positif yang benar **(True Positif)** dan sesedikit mungkin prediksi kelas **False Negative**. Jadi, nanti metric utama yang digunakan adalah `Recall`.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r"C:\Users\faisa\Desktop\DataSciencePurwadhika\Modul3ML\HotelBookingDemand\data\raw\data_hotel_booking_demand.csv")
data.shape

(83573, 11)